## 多个feas_iti且最后一趟列车id相同的乘客

|           |    rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
|:----------|-------:|-------:|-----------:|-------:|---------:|---------:|----------:|
|93194730   | 1525794    |   2 | 1076103701  |     2 | 10301630   |  65071   |   65670|
|8610761    |  119014    |   6 | 1101102701  |     3 | 11002833   |  30045   |   30791|
|18578209   |  230428    |   6 | 1053108503  |     2 | 10401849   |  29715   |   30255|
|72855273   | 1321497    |   3 | 1110112802  |     1 | 10401844   |  60803   |   61537|
|105488240  | 1781873    |   2 | 1028106602  |     2 | 10401738   |  69396   |   69625|
|90908613   | 1485498    |   2 | 1023112302  |     2 | 10702370   |  64374   |   64780|
|91388272   | 1494286    |   1 | 1104111501  |     2 | 10702247   |  65070   |   65278|
|53621559   |  957712    |   6 | 1121102503  |     3 | 10401786   |  49174   |   50141|
|19767473   |  243641    |   1 | 1081103901  |     2 | 10702323   |  29509   |   31141|
|107978876  | 1880069    |   2 | 1129107002  |     1 | 10201226   |  73278   |   74154|
|106615427  | 1817923    |   1 | 1027110801  |     2 | 10301362   |  71471   |   73514|
|76737269   | 1359494    |   1 | 1099102101  |     2 | 10301421   |  64071   |   65266|
|100422001  | 1661221    |   5 | 1061108702  |     1 | 10100270   |  66558   |   66886|
|2408620    |   41409    |   2 | 1111106502  |     1 | 10401886   |  25727   |   25836|
|101105912  | 1675451    |   1 | 1110101301  |     1 | 10401914   |  66944   |   67031|
|92513072   | 1514419    |   3 | 1097110302  |     3 | 10200839   |  65302   |   65408|
|81121188   | 1412555    |   1 | 1061109504  |     2 | 10201199   |  63371   |   63480|
|108069499  | 1886032    |   1 | 1032108201  |     1 | 10200950   |  73568   |   75192|
|73737366   | 1331466    |   1 | 1086110901  |     2 | 10702414   |  61616   |   62107|
|48046191   |  811513    |   3 | 1076103602  |     3 | 10301265   |  42747   |   43084|

In [3]:
import pandas as pd
from src import config
config.load_config()
from src.utils import read_
from src.globals import get_platform
from src.walk_time_dis import filter_transfer_time_from_assigned, get_transfer_platform_ids_from_path, \
    map_platform_id_to_platform_uid

path2trans = get_transfer_platform_ids_from_path()  # path_id, seg_id, node1, node2, transfer_type
df_p2t = pd.DataFrame(path2trans, columns=["path_id", "seg_id", "node1", "node2", "transfer_type"])
df_tt = filter_transfer_time_from_assigned()  # path_id, seg_id, alight_ts, transfer_time

p_id2p_uid = map_platform_id_to_platform_uid()

# map platform_id to platform_uid, and then get unique tuple with (min_, max_)
df_p2t["p_uid1"] = df_p2t["node1"].map(p_id2p_uid)
df_p2t["p_uid2"] = df_p2t["node2"].map(p_id2p_uid)
df_p2t["p_uid_min"] = df_p2t[["p_uid1", "p_uid2"]].min(axis=1)
df_p2t["p_uid_max"] = df_p2t[["p_uid1", "p_uid2"]].max(axis=1)
# df_p2t = df_p2t[["path_id", "seg_id", "p_uid_min", "p_uid_max", "transfer_type"]]
df_p2t

[INFO] Reading file: results\path\path.pkl
[INFO] Reading 1 versioned files: results\trajectory\assigned.pkl
[INFO] Reading file: results\network\node_info.pkl


,path_id,seg_id,node1,node2,transfer_type,p_uid1,p_uid2,p_uid_min,p_uid_max
0,1001100501,1,101220,107211,egress-entry,10122,10721,10122,10721
1,1001100501,2,107461,102291,egress-entry,10746,10229,10229,10746
2,1001100502,1,101250,104311,egress-entry,10125,10431,10125,10431
3,1001100502,2,104291,102321,platform_swap,102321,102321,102321,102321
4,1001100503,1,101260,102351,egress-entry,10126,10235,10126,10235
...,...,...,...,...,...,...,...,...,...
117992,1136113408,3,101221,107210,egress-entry,10122,10721,10122,10721
117993,1136113409,1,110211,107400,egress-entry,103371,107400,103371,107400
117994,1136113409,2,107440,104260,egress-entry,10744,10426,10426,10744
117995,1136113409,3,104370,107291,egress-entry,10437,10729,10437,10729


In [4]:
df = pd.merge(df_tt, df_p2t, on=["path_id", "seg_id"], how="left")
df

,path_id,seg_id,alight_ts,transfer_time,node1,node2,transfer_type,p_uid1,p_uid2,p_uid_min,p_uid_max
0,1101103801,1,21294,307,102290,107461,egress-entry,10229,10746,10229,10746
1,1082111601,1,20999,427,102421,107310,egress-entry,10242,10731,10242,10731
2,1082105801,1,20999,427,102421,107310,egress-entry,10242,10731,10242,10731
3,1082111501,1,20999,427,102421,107310,egress-entry,10242,10731,10242,10731
4,1082105801,1,20999,427,102421,107310,egress-entry,10242,10731,10242,10731
...,...,...,...,...,...,...,...,...,...,...,...
133303,1018101201,1,84544,173,101321,107371,egress-entry,10132,10737,10132,10737
133304,1133110102,1,83912,347,104290,102321,egress-entry,102320,102321,102320,102321
133305,1033109601,1,84324,237,102351,101261,egress-entry,10235,10126,10126,10235
133306,1042111601,1,84544,173,101321,107371,egress-entry,10132,10737,10132,10737


In [20]:
from src.walk_time_dis import reject_outlier
from matplotlib import pyplot as plt

for (p_uid1, p_uid2), df_ in df[df["transfer_type"] == "egress-entry"].groupby(["p_uid_min", "p_uid_max"])["transfer_time"]:
    data  = df_.values
    print(p_uid1, p_uid2, data.size, end=" -> ")
    data = reject_outlier(data, abs_max=500)
    print(data.size)
    plt.hist(data)
    plt.show()
    plt.close()

10122 10721 2077 -> 2075
10125 10431 3839 -> 3839
10126 10235 10844 -> 10832
10129 10333 8900 -> 8875
10132 10737 15332 -> 15243
10229 10746 7979 -> 7978
10242 10731 8209 -> 8188
10325 10722 6747 -> 6706
10329 10433 8315 -> 8240
10426 10744 13670 -> 13507
10437 10729 3928 -> 3909
101410 101411 94 -> 94
102320 102321 236 -> 236
102360 103300 3379 -> 3371
102360 103301 3643 -> 3641
103370 107400 3227 -> 3211
103370 110210 1608 -> 1608
103371 107400 5828 -> 5818
107400 110210 2145 -> 2145


In [19]:
from matplotlib import pyplot as plt

for (p_uid1, p_uid2), df_ in df[df["transfer_type"] == "platform_swap"].groupby(["p_uid_min", "p_uid_max"])["transfer_time"]:
    data  = df_.values
    print(p_uid1, p_uid2, data.size, end=" -> ")
    data = reject_outlier(data, abs_max=500)
    print(data.size)
    plt.hist(data)
    plt.show()
    plt.close()

102320 102320 7780 -> 7746
102321 102321 7896 -> 7844
103370 103371 7632 -> 7571


In [16]:
df[(df['p_uid_min'] == 103370) & (df['p_uid_max'] == 103371)]

,path_id,seg_id,alight_ts,transfer_time,node1,node2,transfer_type,p_uid1,p_uid2,p_uid_min,p_uid_max
434,1136102201,1,23350,60,110211,103371,platform_swap,103371,103370,103370,103371
671,1074100601,1,23860,48,110211,103371,platform_swap,103371,103370,103370,103371
754,1060106201,1,24350,57,110211,103371,platform_swap,103371,103370,103370,103371
761,1060106201,1,24350,57,110211,103371,platform_swap,103371,103370,103370,103371
781,1060105201,1,24350,57,110211,103371,platform_swap,103371,103370,103370,103371
...,...,...,...,...,...,...,...,...,...,...,...
133025,1027102201,1,83230,220,110211,103371,platform_swap,103371,103370,103370,103371
133044,1027103301,1,83230,220,110211,103371,platform_swap,103371,103370,103370,103371
133082,1074100601,1,83230,220,110211,103371,platform_swap,103371,103370,103370,103371
133236,1095103301,1,83230,220,110211,103371,platform_swap,103371,103370,103370,103371
